In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# ENV
python -m venv data-viz-env
data-viz-env\Scripts\activate
pip install -r requirements-dev.txt
jupyter notebook

In [8]:

# Base directory configuration
BASE_PC_DIR = Path(r'C:\Users\Work\Documents\GitHub\UNI\FDS02Q001\data_viz')
BASE_NB_DIR = Path(r'C:\Users\g.evola\repo\UNI\FDS02Q001\data_viz')

# Choose based on pc used
BASE_DIR = BASE_NB_DIR
DATA_DIR = BASE_DIR / 'datasets' / 'prison_statistics'

## 1 - Preliminary analysis
### Data
* geo (Country) -> names of european countries
* TIME_PERIOD (Year) -> from 2014 to 2022
* indic_cr (indicators) -> Actual number of persons held in prison or Official prison capacity - persons
* unit (unit of measure) -> Number or Per hundred thousand inhabitants(%)
* OBS_VALUE (value of the indicator in the specified unit of measurement)

In [32]:

# Dataset loading
df = pd.read_csv(DATA_DIR / 'crim_pris_cap$defaultview_linear.csv', sep=',')


# Cast numeric datatype
df['YEAR'] = pd.to_numeric(df['TIME_PERIOD'], errors='coerce')
df['VALUE'] = pd.to_numeric(df['OBS_VALUE'], errors='coerce')

# Keep only years up to 2022 (complete data)
df = df[df["YEAR"] <= 2022].copy()

In [39]:
# --- 1) ABSOLUTE NUMBERS ---------------------------------------------
df_num = df[df["unit"] == "Number"].copy()

pivot_num = df_num.pivot_table(
    index=["geo", "YEAR"],
    columns="indic_cr",
    values="VALUE",
    aggfunc="first"
)

pivot_num = pivot_num.rename(columns={
    "Actual number of persons held in prison": "PRISONERS_NUM",
    "Official prison capacity - persons": "CAPACITY_NUM"
})

# keep only rows with both values
pivot_num = pivot_num.dropna(subset=["PRISONERS_NUM", "CAPACITY_NUM"])

# Absolute occupancy rate (%)
pivot_num["OCC_ABS"] = pivot_num["PRISONERS_NUM"] / pivot_num["CAPACITY_NUM"] * 100


# --- 2) PER 100K INHABITANTS ----------------------------------------
df_100k = df[df["unit"] == "Per hundred thousand inhabitants"].copy()

pivot_100k = df_100k.pivot_table(
    index=["geo", "YEAR"],
    columns="indic_cr",
    values="VALUE",
    aggfunc="first"
)

pivot_100k = pivot_100k.rename(columns={
    "Actual number of persons held in prison": "PRISONERS_100K",
    "Official prison capacity - persons": "CAPACITY_100K"
})

pivot_100k = pivot_100k.dropna(subset=["PRISONERS_100K", "CAPACITY_100K"])

pivot_100k["OCC_PER_100K"] = (
    pivot_100k["PRISONERS_100K"] / pivot_100k["CAPACITY_100K"] * 100
)


# --- 3) MERGE BOTH VERSIONS -----------------------------------------
occupancy = pivot_num.join(pivot_100k, how="inner")

# Round occupancy metrics to 2 decimal places
occupancy["OCC_ABS"] = occupancy["OCC_ABS"].round(2)
occupancy["OCC_PER_100K"] = occupancy["OCC_PER_100K"].round(2)

occupancy_df = occupancy.reset_index()

print("Final occupancy dataset shape:", occupancy_df.shape)
print("Years:", occupancy_df["YEAR"].min(), "→", occupancy_df["YEAR"].max())
print("Countries:", occupancy_df["geo"].nunique())
print("\nColumns:", occupancy_df.columns.tolist())


Final occupancy dataset shape: (308, 8)
Years: 2014 → 2022
Countries: 39

Columns: ['geo', 'YEAR', 'PRISONERS_NUM', 'CAPACITY_NUM', 'OCC_ABS', 'PRISONERS_100K', 'CAPACITY_100K', 'OCC_PER_100K']


In [40]:
# Save for later analysis & visualizations
occupancy_df.to_csv(DATA_DIR / "prison_occupancy_dual_2014_2022.csv", index=False)
print("\n Saved 'prison_occupancy_dual_2014_2024.csv'")


 Saved 'prison_occupancy_dual_2014_2024.csv'


In [36]:
occ_2022 = occupancy_df[occupancy_df["YEAR"] == 2022].copy()

print("Rows 2022:", occ_2022.shape)
print(occ_2022.head())

# opzionale: salva per Datawrapper / Plotly
occ_2022.to_csv(DATA_DIR / "prison_occupancy_2022.csv", index=False)
print("Saved 'prison_occupancy_2022.csv'")


Rows 2022: (35, 8)
indic_cr       geo  YEAR  PRISONERS_NUM  CAPACITY_NUM  OCC_ABS  \
8          Albania  2022         5060.0        5657.0    89.45   
17         Austria  2022         8993.0        8473.0   106.14   
23         Belgium  2022        11050.0        9703.0   113.88   
32        Bulgaria  2022         5577.0        8161.0    68.34   
40         Croatia  2022         4091.0        3927.0   104.18   

indic_cr  PRISONERS_100K  CAPACITY_100K  OCC_PER_100K  
8                 181.13         202.50         89.45  
17                100.16          94.37        106.14  
23                 95.11          83.52        113.88  
32                 81.55         125.89         64.78  
40                105.92         101.68        104.17  
Saved 'prison_occupancy_2022.csv'


In [ ]:
## 2 - How many EU/European countries exceed 100% prison occupancy rate in the 2023?

In [25]:
# Load the processed occupancy dataset
occ = pd.read_csv(DATA_DIR / "prison_occupancy_dual_2014_2024.csv")

# 1. Identify latest available year
latest_year = occ["YEAR"].max()
print(f"📅 Latest available year: {latest_year}")

latest = occ[occ["YEAR"] == latest_year].copy()

# 2. Count countries above 100% (absolute occupancy)
over_100 = latest[latest["OCC_ABS"] > 100]

num_countries_over_100 = over_100["geo"].nunique()
total_countries = latest["geo"].nunique()

print(f"\n🔥 Countries with OCC_ABS > 100% in {latest_year}: {num_countries_over_100} / {total_countries}")
print("\nList of countries (sorted by OCC_ABS):")
print(
    over_100[["geo", "OCC_ABS", "OCC_PER_100K"]]
    .sort_values("OCC_ABS", ascending=False)
    .round(2)
    .to_string(index=False)
)


📅 Latest available year: 2023

🔥 Countries with OCC_ABS > 100% in 2023: 13 / 34

List of countries (sorted by OCC_ABS):
          geo  OCC_ABS  OCC_PER_100K
Liechtenstein   218.18        218.18
       France   122.90        122.89
        Italy   119.08        119.08
      Belgium   113.22        113.23
       Sweden   112.59        112.59
      Croatia   109.70        109.70
      Romania   109.56        109.56
      Austria   107.45        107.45
     Slovenia   106.91        106.93
      Ireland   104.81        104.81
      Denmark   102.44        102.44
       Greece   100.78        100.78
      Finland   100.14        100.15


In [ ]:
# 3 - Which countries have been most successful in maintained occupancy rates below capacity in the 2023?

In [27]:
occ = pd.read_csv(DATA_DIR / "prison_occupancy_dual_2014_2024.csv")

year = 2023  # focus year
subset = occ[occ["YEAR"] == year].copy()

# Sort by absolute occupancy rate (ascending = best)
best_2023 = subset.sort_values("OCC_ABS", ascending=True)

top10_best = best_2023.head(10)
print(f"✅ Top 10 countries with the lowest prison occupancy in {year}:")
print(top10_best[["geo", "OCC_ABS", "OCC_PER_100K"]].round(2).to_string(index=False))


✅ Top 10 countries with the lowest prison occupancy in 2023:
       geo  OCC_ABS  OCC_PER_100K
   Estonia    56.16         56.16
Luxembourg    60.80         60.80
  Bulgaria    67.71         67.71
 Lithuania    67.81         67.82
    Latvia    67.83         67.83
     Malta    70.59         70.59
     Spain    73.76         73.76
Montenegro    78.47         78.47
   Germany    81.83         81.83
  Slovakia    84.13         84.13


In [ ]:
# 4 - What are the regional differences (North vs South Europe) in prison overcrowding?

In [29]:
occ = pd.read_csv(DATA_DIR / "prison_occupancy_dual_2014_2024.csv")

north = [
    "Norway","Sweden","Finland","Denmark","Iceland",
    "Ireland","United Kingdom","Northern Ireland (UK) (NUTS 2021)",
    "Netherlands","Estonia","Latvia","Lithuania",
    "Germany","Austria","France","Belgium","Luxembourg","Switzerland",
    "Poland","Czechia","Slovakia", "Liechtenstein"
]

south = [
    "Italy","Spain","Portugal","Greece","Malta","Cyprus",
    "Croatia","Slovenia","Romania","Bulgaria","Hungary","Serbia",
    "Bosnia and Herzegovina", "Montenegro", "North Macedonia",
    "Albania", "Kosovo*"
]

def assign_region(country):
    if country in north:
        return "North"
    if country in south:
        return "South"
    return "Other"

occ["REGION"] = occ["geo"].apply(assign_region)

occ.to_csv("prison_occupancy_with_regions.csv", index=False)
print("✅ Regions assigned")


✅ Regions assigned


In [30]:
latest_year = occ["YEAR"].max()
print("Latest year:", latest_year)

latest = occ[occ["YEAR"] == latest_year]

region_stats = (
    latest.groupby("REGION")["OCC_ABS"]
    .agg(["mean","min","max","count"])
    .round(2)
)

print("\n📊 North vs South in", latest_year)
print(region_stats.loc[["North","South"]])


Latest year: 2023

📊 North vs South in 2023
         mean    min     max  count
REGION                             
North   97.53  56.16  218.18     19
South   93.32  67.71  119.08     14


In [ ]:
# 5 - Are their trends improving, stable, or worsening over the 2014–2023 period, compared to the overall European average?

In [31]:
occ = pd.read_csv(DATA_DIR / "prison_occupancy_dual_2014_2024.csv")

# Focus on 2014–2023 (exclude 2024 if incomplete)
trend_df = occ[(occ["YEAR"] >= 2014) & (occ["YEAR"] <= 2023)].copy()

# 1) European average per year
eu_avg = (
    trend_df.groupby("YEAR")["OCC_ABS"]
    .mean()
    .reset_index()
    .rename(columns={"OCC_ABS": "EU_OCC_ABS"})
)

print("EU average occupancy 2014–2023:")
print(eu_avg)

# 2) Simple trend metric per country: difference 2014 → 2023
def compute_trend(group):
    group = group.sort_values("YEAR")
    first = group[group["YEAR"] == group["YEAR"].min()]["OCC_ABS"].iloc[0]
    last = group[group["YEAR"] == group["YEAR"].max()]["OCC_ABS"].iloc[0]
    return pd.Series({
        "OCC_2014": first,
        "OCC_2023": last,
        "DELTA_14_23": last - first
    })

country_trends = (
    trend_df.groupby("geo")
    .apply(compute_trend)
    .reset_index()
)

print("\nCountry trends 2014–2023 (sample):")
print(country_trends.head())

# 3) Classify trend
def classify(delta, threshold=2):
    if delta > threshold:
        return "Worsening"
    elif delta < -threshold:
        return "Improving"
    else:
        return "Stable"

country_trends["TREND_CLASS"] = country_trends["DELTA_14_23"].apply(classify)

print("\nTrend classification:")
print(
    country_trends[["geo","OCC_2014","OCC_2023","DELTA_14_23","TREND_CLASS"]]
    .sort_values("DELTA_14_23", ascending=False)
    .round(2)
    .to_string(index=False)
)


EU average occupancy 2014–2023:
   YEAR  EU_OCC_ABS
0  2014  104.898065
1  2015  101.526562
2  2016  105.170937
3  2017  105.649706
4  2018  100.401622
5  2019   99.638889
6  2020   94.572500
7  2021   95.898857
8  2022  100.115714
9  2023   95.829706

Country trends 2014–2023 (sample):
        geo  OCC_2014  OCC_2023  DELTA_14_23
0   Albania    124.30     87.60       -36.70
1   Austria    100.30    107.45         7.15
2   Belgium    110.47    113.22         2.75
3  Bulgaria    100.00     67.71       -32.29
4   Croatia     93.56    109.70        16.14

Trend classification:
                              geo  OCC_2014  OCC_2023  DELTA_14_23 TREND_CLASS
                           Cyprus    113.96    225.94       111.98   Worsening
                           Sweden     89.68    112.59        22.91   Worsening
                          Denmark     86.27    102.44        16.17   Worsening
                          Croatia     93.56    109.70        16.14   Worsening
                        

C:\Users\g.evola\AppData\Local\Temp\ipykernel_4336\1565128498.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_trend)


In [ ]:
# 6 - mini

In [42]:
occ = pd.read_csv(DATA_DIR / "prison_occupancy_dual_2014_2022.csv")

countries_focus = ["Iceland", "Malta", "Liechtenstein", "Cyprus"]

focus_trends = occ[occ["geo"].isin(countries_focus)].copy()
focus_trends = focus_trends.sort_values(["geo", "YEAR"])

print(focus_trends[["geo","YEAR","OCC_ABS","OCC_PER_100K"]])

# opzionale: salva per Plotly / grafico
focus_trends.to_csv(DATA_DIR / "prison_trends_2014_2022_focus4.csv", index=False)
print("Saved 'prison_trends_2014_2022_focus4.csv'")


               geo  YEAR  OCC_ABS  OCC_PER_100K
41          Cyprus  2015   113.96        113.96
42          Cyprus  2017   112.12        112.13
43          Cyprus  2018   114.00        113.99
44          Cyprus  2019   139.78        139.78
45          Cyprus  2020   110.68        110.68
46          Cyprus  2021   146.41        146.42
47          Cyprus  2022   225.94        225.92
123        Iceland  2014    99.35         99.37
124        Iceland  2015    93.92         93.93
125        Iceland  2016    82.86         82.85
126        Iceland  2018    83.95         83.95
127        Iceland  2019    89.27         89.27
128        Iceland  2020    80.57         80.57
129        Iceland  2021    78.45         78.44
130        Iceland  2022    76.24         76.24
161  Liechtenstein  2014   265.00        264.99
162  Liechtenstein  2015   265.00        265.02
163  Liechtenstein  2016   360.00        360.01
164  Liechtenstein  2017   365.00        364.97
165  Liechtenstein  2018   315.00       

In [ ]:
# 7 - PLOTLY TIME SERIES (Kaleido)

In [ ]:
# Load trends data
plot_data = pd.read_csv(DATA_DIR / 'prison_mini_trends_2015_2022.csv')

print(f"📊 Data loaded: {len(plot_data)} rows")

# Create professional line chart
fig = px.line(plot_data, 
              x='YEAR', 
              y='OCC_ABS',
              color='geo',
              markers=True,
              title="Prison Occupancy Trends 2014-2022: European countries with prison occupancy < 1000",
              labels={'OCC_ABS': 'Occupancy Rate (%)', 
                     'YEAR': 'Year'})

fig.update_layout(
    height=500, 
    width=1000, 
    font_size=14,
    title_font_size=16,
    legend_title="Countries",
    showlegend=True
)

# Overcrowding threshold (100%)
fig.add_hline(y=100, line_dash="dash", line_color="red", 
              annotation_text="OVERCROWDING THRESHOLD", 
              annotation_position="top right")

print("✅ Chart created successfully!")

# EXPORT (multiple formats - handles all Kaleido issues)

try:
    fig.write_html(DATA_DIR / "plotly_prison_trends.html")
    print("✅ HTML (interactive) saved")
except:
    print("⚠️ HTML failed")
    
try:
    fig.write_image(DATA_DIR / "plotly_prison_trends.png", scale=3, width=1800, height=900)
    print("✅ PNG (high-res PDF) saved")
except:
    print("⚠️ PNG failed → Use HTML screenshot")

try:
    fig.write_image(DATA_DIR / "plotly_prison_trends.svg")
    print("✅ SVG (vector) saved")
except:
    print("⚠️ SVG failed → Use PNG/HTML")

fig.show()
